# A Quick Introduction to FinOL

This Jupyter notebook goes through the basic usage of FinOL.

- Install FinOL
- Complete the whole data-training-testing process involves
  - Read train/valid/test data
  - Define and optimize model
  - Evaluate model

## Install FinOL and additional dependencies

``FinOL`` can be installed via `pip` or from the master branch of Git repository.
Most of the dependencies required by ``FinOL`` are automatically installed when using
 the provided pyproject.toml file. HHowever, specific dependencies like ``TA-Lib``
 require separate installation as it is a library with C/C++ bindings.

In the code snippet below, you can see the method to install ``TA-Lib`` in a Colab environment.
For users running ``FinOL`` on Windows, Linux, or Mac, please refer to our official documentation for
detailed installation instructions. The documentation outlines how to install ``FinOL`` completely, including ``TA-Lib``.

In [1]:
# install finol and most dependencies
!pip install finol

# install ta-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar xvzf ta-lib-0.4.0-src.tar.gz
import os
os.chdir('ta-lib')
!./configure --prefix=/usr
!make
!make install
os.chdir('../')
!pip install TA-Lib

In [ ]:
import finol

finol.__version__

## Quickstart: command line usage

Before running the above commands, users can first configure some parameters through the config file to customize the usage according to their needs. For example, setting the device to CPU, selecting a different dataset, adjusting the data preprocessing parameters, and choosing a different model, etc. The specific configuration method is as follows:

In [ ]:
from finol.utils import load_config, update_config

config = load_config()
config["DEVICE"] = "cpu"
config["DATASET_NAME"] = "DJIA"
config["DATA_AUGMENTATION_CONFIG"]["WINDOW_DATA"]["WINDOW_SIZE"] = 15
config["SCALER"] = "WindowMinMaxScaler"
config["MODEL_NAME"] = "DNN"
config["TUNE_PARAMETERS"] = True
update_config(config)

In [2]:
from finol.data_layer.DatasetLoader import DatasetLoader
from finol.optimization_layer.ModelTrainer import ModelTrainer
from finol.evaluation_layer.ModelEvaluator import ModelEvaluator


load_dataset_output = DatasetLoader().load_dataset()
train_model_output = ModelTrainer(load_dataset_output).train_model()
evaluate_model_output = ModelEvaluator(load_dataset_output, train_model_output).evaluate_model()

{'train_loader': <torch.utils.data.dataloader.DataLoader at 0x2e77fea0d30>,
 'val_loader': <torch.utils.data.dataloader.DataLoader at 0x2e77feb57f0>,
 'test_loader': <torch.utils.data.dataloader.DataLoader at 0x2e77feb5af0>,
 'NUM_TRAIN_PERIODS': 3293,
 'NUM_VAL_PERIODS': 1130,
 'NUM_TEST_PERIODS': 1131,
 'NUM_ASSETS': 26,
 'NUM_FEATURES_AUGMENTED': 1430,
 'NUM_FEATURES_ORIGINAL': 143,
 'DETAILED_NUM_FEATURES': {'OHLCV_FEATURES': 5,
  'OVERLAP_FEATURES': 20,
  'MOMENTUM_FEATURES': 40,
  'VOLUME_FEATURES': 3,
  'CYCLE_FEATURES': 7,
  'PRICE_FEATURES': 4,
  'VOLATILITY_FEATURES': 3,
  'PATTERN_FEATURES': 61},
 'WINDOW_SIZE': 10,
 'OVERALL_FEATURE_LIST': ['OHLCV_FEATURES',
  'OVERLAP_FEATURES',
  'MOMENTUM_FEATURES',
  'VOLUME_FEATURES',
  'CYCLE_FEATURES',
  'PRICE_FEATURES',
  'VOLATILITY_FEATURES',
  'PATTERN_FEATURES'],
 'DETAILED_FEATURE_LIST': ['OPEN',
  'HIGH',
  'LOW',
  'CLOSE',
  'VOLUME',
  'BBANDS_UPPER',
  'BBANDS_MIDDLE',
  'BBANDS_LOWER',
  'DEMA',
  'EMA',
  'HT_TRENDLINE'

**Training the Model**

We next define a custom loss function called ``criterion`` for the following training process. In ``FinOL``, users have the flexibility to define their own custom loss functions.  This allows users to tailor the training process to their specific needs.  For example, users may want to define a custom loss function that includes risk measurement.

Following that, we implement define the model architecture with the LSTM layer, specifying the input feature and output sizes based on the dataset characteristics obtained from the data loading stage.  The model is then moved to the designated device.

An Adam optimizer is then chosen for training the network weights. In ``FinOL``, a comprehensive range of over 40 optimizers is available.

The core training loop iterates over mini-batches from the training dataloader, performs forward, backward and optimization steps at each turn.  Validation loss is also computed periodically to select the best performing model checkpoint for later deployment.

This covers the essential components involved in setting up and executing a deep learning workflow on financial data within the PyTorch framework as implemented in our tutorial.  We are now ready to kick off model training and analysis of results.